In [ ]:
# The objective of this script is to make visualisations that shows for each read of the query file the quality of 
# its alignament between against the different refferences. Also, this script will make barplots that shows the the
# quality of the reads mapped in each reference. And plots that shows the quality and quantity of the 
# reference sequences that were mapped. 
# 
# 1 Line plot with mapping qualities of each query sequences against the different references. 
#   sorted by the qualitiess of the best aligned reference
# 2 Barplot showing the percentage of reads mapped in each references, and the quality of the mapped sequences
# 3 Barplot showing only for the sequences mapped the quality of its alignament
# 4 Try to use different metrics for check quality (Identity, Bitscore)     
# 5 Subidive the alignament table, grouping the best read alignamentet to each reference sequencs

In [1]:
import pandas as pd
import re
import os
import numpy as np

os.getcwd()

'/home/jpereira/OEs/OE1/Scripts'

In [2]:
from Bio import SeqIO

# Replace 'your_fasta_file.fasta' with the path to your FASTA file
fasta_file = '../toxo_reads/toxo1.trimmedReads.fasta'

# Parse the FASTA file and extract the lengths
total_len = 0
num_records = 0
query_lens = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    total_len += len(record.seq)
    num_records += 1
    query_lens[record.id] = len(record.seq)

print(f'Total reads length: {total_len}')
print(f'Sequence number: {num_records}')

Total reads length: 1599549811
Sequence number: 128876


In [3]:
import pandas as pd
import re
import os
import numpy as np

# Define the path to your PAF file
paf_file1_path = '../Data/map_contamination/TGVEG.paf'
paf_file2_path = '../Data/map_contamination/TGA4.paf'
paf_file3_path = '../Data/map_contamination/TgRH_pasteur.paf'
paf_file4_path = '../Data/map_contamination/Chlorocebus_sabaeus.paf'

paf_files_path_list = [paf_file1_path, paf_file2_path, paf_file3_path, paf_file4_path]

# Define column names based on the PAF specification
column_names = [
    'query_name', 'query_length', 'query_start', 'query_end',
    'relative_strand', 'target_name', 'target_length', 'target_start',
    'target_end', 'num_matching_residues', 'alignment_block_length', 'mapping_quality'
, 'Undf1','Undf2','Undf3', 'Undf4', 'Undf5', 'Undf6']

# Load the PAF file into a DataFrame
paf_df = pd.DataFrame()
for path in paf_files_path_list:

    df = pd.read_csv(path, sep='\t', header=None, names=column_names)
    file_name = os.path.basename(path)
    org_name = re.sub('.[^.]*$', '', file_name)
    
    df['org_name'] = org_name

    select_quality = df['mapping_quality'] > 45
    select_cover = (df['query_end'] - df['query_start']) / df['query_length'] > 0.9

    

    select = select_cover & select_quality
    df = df[select]

    df['identity'] = df['num_matching_residues'] / df['query_length']

    # If a query has more than two alignaments (for the same reference),
    # select only the one with the greatest identity
    max_query_ident = df.groupby('query_name')['identity'].idxmax() 
    df = df.loc[max_query_ident.values]

    paf_df = pd.concat([paf_df, df], axis=0)

max_query_ident

print(paf_df.shape[0])
paf_df.head(3)


# Display the first few rows of the DataFrame



330996


,query_name,query_length,query_start,query_end,relative_strand,target_name,target_length,target_start,target_end,num_matching_residues,alignment_block_length,mapping_quality,Undf1,Undf2,Undf3,Undf4,Undf5,Undf6,org_name,identity
179694,000230c9-5c46-4db0-9fff-06db681304c3,12155,9,12154,+,KI544497.1,4970800,702548,714691,10886,12207,60,tp:A:P,cm:i:1873,s1:i:10861,s2:i:0,dv:f:0.0031,rl:i:153,TGVEG,0.895599
135270,00028b49-baf3-4508-bc07-37dab84feba1,5096,6,5089,+,KI544492.1,6950490,1996854,2001906,4432,5114,60,tp:A:P,cm:i:746,s1:i:4410,s2:i:0,dv:f:0.0034,rl:i:190,TGVEG,0.869702
177654,0003aff6-0ec3-44cc-b1db-0960db1d9132,14828,4,14823,-,KI544497.1,4970800,2401697,2416527,13757,14833,60,tp:A:P,cm:i:2409,s1:i:13752,s2:i:0,dv:f:0.0016,rl:i:15,TGVEG,0.927772


In [4]:
paf_df.index = paf_df.reset_index().index
id_max_ident = paf_df.groupby('query_name')['identity'].idxmax()

query_max_iden_org_df = paf_df.loc[id_max_ident.values, ['query_name','org_name']]
query_max_iden_org_df = query_max_iden_org_df.rename(columns={ 'org_name' : 'max_iden_org_name'})

paf_max_iden_df = pd.merge(paf_df, query_max_iden_org_df, on='query_name', how='left')
paf_max_iden_df.sort_values('query_name').head(3)

,query_name,query_length,query_start,query_end,relative_strand,target_name,target_length,target_start,target_end,num_matching_residues,...,mapping_quality,Undf1,Undf2,Undf3,Undf4,Undf5,Undf6,org_name,identity,max_iden_org_name
0,000230c9-5c46-4db0-9fff-06db681304c3,12155,9,12154,+,KI544497.1,4970800,702548,714691,10886,...,60,tp:A:P,cm:i:1873,s1:i:10861,s2:i:0,dv:f:0.0031,rl:i:153,TGVEG,0.895599,TgRH_pasteur
108515,000230c9-5c46-4db0-9fff-06db681304c3,12155,9,12154,-,CM002047.1,7094428,5964721,5976866,10904,...,60,tp:A:P,cm:i:1879,s1:i:10880,s2:i:0,dv:f:0.0031,rl:i:83,TGA4,0.897079,TgRH_pasteur
218614,000230c9-5c46-4db0-9fff-06db681304c3,12155,9,12154,-,CM065588.1,7235803,6100174,6112342,12069,...,60,tp:A:P,cm:i:2255,s1:i:12064,s2:i:0,dv:f:0.0002,rl:i:113,TgRH_pasteur,0.992925,TgRH_pasteur


In [5]:
paf_max_iden_df['identity'] = paf_max_iden_df['identity'].round(2)

In [6]:
group_org_idn_seq_len = paf_max_iden_df.groupby(['org_name', 'identity'])['query_length'].sum()
len_id_org_df = pd.DataFrame(group_org_idn_seq_len)
select_p = len_id_org_df/total_len > 0.05/100

#len_id_org_df = len_id_org_df[select_p].dropna()

#len_id_org_df.index.get_level_values('org_name').unique()
len_id_org_df

query_length
org_name            identity              
Chlorocebus_sabaeus 0.13              7408
                    0.14              4374
                    0.15              3829
                    0.24              1802
                    0.25              1440
...                                    ...
TgRH_pasteur        0.96          39332855
                    0.97          94139272
                    0.98         248560854
                    0.99         676733403
                    1.00         365502443

[288 rows x 1 columns]

In [ ]:
## Make bins of 0.1% the total length of sequences
# 1: Add to each 'bin' query_length values until is full.
# 2: Calculate the identity average of each mean, with it correspondeing quartiles
#    -Divide all query lengths by the smallest query_length, round the numbers to 3 decimal places
#    -Multiply each query_length by its corresponding identity.
#    -Divide all by the bin ocupied size dividev by the smallest query_length
# 3: Move to the next bin
# 4: Add what is left of the last value of the previus bin to the current bin
# 5: Repeat
#


In [7]:
# Extract unique 'org_name' values
unique_org_names = group_org_idn_seq_len.index.get_level_values('org_name').unique()

print(unique_org_names)


Index(['Chlorocebus_sabaeus', 'TGA4', 'TGVEG', 'TgRH_pasteur'], dtype='object', name='org_name')


In [8]:
# Assuming unique_org_names and group_org_idn_seq_len are already defined

for org in unique_org_names:
    print(f"Organization: {org}")
    
    # Access the data corresponding to the specific org_name
    org_data = group_org_idn_seq_len.loc[org]

    
    print(org_data/total_len)
    print("\n")  # To add a newline between each organization's output



Organization: Chlorocebus_sabaeus
identity
0.13    4.631303e-06
0.14    2.734519e-06
0.15    2.393799e-06
0.24    1.126567e-06
0.25    9.002533e-07
0.29    7.383327e-07
0.30    9.258855e-07
0.35    1.660467e-06
0.59    1.776187e-05
0.62    2.530900e-05
0.64    3.285487e-05
0.65    1.552812e-05
0.67    2.710200e-05
0.68    4.459380e-06
0.69    1.240474e-05
0.70    1.953675e-06
0.71    3.385328e-06
0.72    6.755651e-06
0.74    4.666938e-06
0.75    6.660624e-06
0.76    2.817668e-06
0.77    7.394581e-06
0.78    7.278298e-06
0.79    1.643900e-05
0.80    2.359414e-06
0.81    7.972868e-06
0.82    2.012691e-05
0.83    1.285612e-05
0.84    2.714514e-06
0.86    8.548030e-06
0.87    2.154981e-06
0.88    8.433623e-07
0.90    3.054359e-05
0.91    7.735301e-06
0.92    2.105467e-05
0.93    6.508706e-06
0.94    1.698290e-05
0.95    1.256479e-05
0.96    5.413086e-05
0.97    7.686600e-05
0.98    6.220625e-05
0.99    6.367041e-05
Name: query_length, dtype: float64


Organization: TGA4
identity
0.17    0.

In [9]:
paf_max_iden_df.groupby('org_name')['identity'].mean().sort_values(ascending=False)

org_name
TgRH_pasteur           0.979596
TGVEG                  0.918427
TGA4                   0.910011
Chlorocebus_sabaeus    0.817652
Name: identity, dtype: float64

In [ ]:
paf_max_iden_df.groupby('query_name')[]

In [11]:
df['per_matching_residues'] = df['num_matching_residues']/df['alignment_block_length']

df[['per_matching_residues', 'query_length']].sort_values(by=['query_length','per_matching_residues'], ascending=False)
#df[['per_matching_residues', 'query_length']].sort_values(by=['per_matching_residues'], ascending=False)
#df['mapping_quality'].sort_values( ascending=False)

,per_matching_residues,query_length
96175,0.617884,39550
13567,0.579000,28859
9761,0.568853,27363
13595,0.624966,24838
35963,0.656961,19842
...,...,...
34556,0.938646,1407
349,0.856560,1356
14973,0.865160,1349
96106,0.313707,1181
